In [1]:
# install required libraries
!pip3 install sentence-transformers

     |████████████████████████████████| 71kB 4.9MB/s 
     |████████████████████████████████| 1.9MB 7.5MB/s 
     |████████████████████████████████| 1.2MB 33.2MB/s 
     |████████████████████████████████| 3.2MB 50.0MB/s 
     |████████████████████████████████| 890kB 48.0MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp37-none-any.whl size=103068 sha256=379fa0d768d7c3a5b77b946184880bb0bb9150cbb56a0bdd34c25316e4bd0c6d
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=1e4fadea6f98d8fde14158e302e728e8ff9ef2b1dc8b4a52af1fcd70bf198877
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [2]:
!pip3 install elasticsearch

     |████████████████████████████████| 327kB 7.0MB/s 


In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/BERT-FAQ

/content/drive/MyDrive/BERT-FAQ


In [6]:
!ls

data			     metric.py	  requirements.txt
evaluation.py		     notebook	  reranker.py
faq_bert_finetuning.py	     output	  searcher.py
faq_bert.py		     parser	  shared
hard_negatives_generator.py  __pycache__  training_data_generator.py
indexer.py		     README.md


In [7]:
# import required dependencies
from evaluation import get_relevance_label_df
from shared.utils import load_from_json
from shared.utils import dump_to_json
from shared.utils import make_dirs
from reranker import ReRanker

In [8]:
output_path="data/CovidFAQ/rank_results"

# load user_query ES results from json files
es_output_path = output_path + "/unsupervised"
es_query_by_question = load_from_json(es_output_path + '/es_query_by_question.json')
es_query_by_answer = load_from_json(es_output_path + '/es_query_by_answer.json')
es_query_by_question_answer = load_from_json(es_output_path + '/es_query_by_question_answer.json')
es_query_by_question_answer_concat = load_from_json(es_output_path + '/es_query_by_question_answer_concat.json')

In [9]:
# load test_queries, relevance_label_df for ReRanker
query_answer_pair_filepath = 'data/CovidFAQ/query_answer_pairs.json'
relevance_label_df = get_relevance_label_df(query_answer_pair_filepath)
test_queries = relevance_label_df[relevance_label_df['query_type'] == 'user_query'].question.unique()

In [10]:
test_queries[:10]

array(['What does Corona and Covid mean?',
       'when will the social distancing end? and what is the economic consequent of the pandemic?',
       'How do I go grocery shopping? ', 'what is covid 19',
       "What is COVID-19's definition?", 'Should I stay home ?',
       'Should I practice social distancing? ', 'How did it started',
       'what are the 3 symptoms of covid-19', 'Should I wear a mask?'],
      dtype=object)

In [11]:
# total number of test queries
len(test_queries)

1078

**#################### Triplet Loss ######################**

**Re-ranking results using query_type="user_query"; neg_type="hard"**

In [ ]:
# define variables
query_type="user_query"; neg_type="hard"; version="1.1"; loss_type='triplet'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df
)

In [ ]:
# generate directory structure
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, reranked_output_path + '/bert_query_by_question.json')
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, reranked_output_path + '/bert_query_by_answer.json')
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, reranked_output_path + '/bert_query_by_question_answer.json')
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, reranked_output_path + '/bert_query_by_question_answer_concat.json')
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

2021-03-11 16:05:00 - Generating BERT top-k results ...
2021-03-11 16:05:00 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_hard_user_query_1.1
2021-03-11 16:05:00 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_hard_user_query_1.1
2021-03-11 16:05:19 - Use pytorch device: cuda
100%|██████████| 1078/1078 [10:57<00:00,  1.64it/s]
2021-03-11 16:16:17 - Re-ranking the top-k results ...
2021-03-11 16:16:18 - Generating BERT top-k results ...
2021-03-11 16:16:18 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_hard_user_query_1.1
2021-03-11 16:16:18 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_hard_user_query_1.1
2021-03-11 16:16:19 - Use pytorch device: cuda
100%|██████████| 1078/1078 [19:29<00:00,  1.08s/it]
2021-03-11 16:35:50 - Re-ranking the top-k results ...
2021-03-11 16:35:51 - Generating BERT top-k results ...
2021-03-11 16:35:51 - Load pretrained SentenceTransformer: output/CovidFAQ/models/

**Re-ranking results using query_type="user_query"; neg_type="simple"**

In [ ]:
# define variables
query_type="user_query"; neg_type="simple"; version="1.1"; loss_type='triplet'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df
)

In [ ]:
# generate directory structure
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, reranked_output_path + '/bert_query_by_question.json')
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, reranked_output_path + '/bert_query_by_answer.json')
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, reranked_output_path + '/bert_query_by_question_answer.json')
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, reranked_output_path + '/bert_query_by_question_answer_concat.json')
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

2021-03-11 17:22:28 - Generating BERT top-k results ...
2021-03-11 17:22:28 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_simple_user_query_1.1
2021-03-11 17:22:28 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_simple_user_query_1.1
2021-03-11 17:22:45 - Use pytorch device: cuda
100%|██████████| 1078/1078 [11:03<00:00,  1.62it/s]
2021-03-11 17:33:50 - Re-ranking the top-k results ...
2021-03-11 17:33:51 - Generating BERT top-k results ...
2021-03-11 17:33:51 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_simple_user_query_1.1
2021-03-11 17:33:51 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_simple_user_query_1.1
2021-03-11 17:33:52 - Use pytorch device: cuda
100%|██████████| 1078/1078 [19:43<00:00,  1.10s/it]
2021-03-11 17:53:37 - Re-ranking the top-k results ...
2021-03-11 17:53:39 - Generating BERT top-k results ...
2021-03-11 17:53:39 - Load pretrained SentenceTransformer: output/CovidFAQ

**Re-ranking results using query_type="faq"; neg_type="hard"**

In [ ]:
# define variables
query_type="faq"; neg_type="hard"; version="1.1"; loss_type='triplet'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df
)

In [ ]:
# generate directory structure
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, reranked_output_path + '/bert_query_by_question.json')
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, reranked_output_path + '/bert_query_by_answer.json')
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, reranked_output_path + '/bert_query_by_question_answer.json')
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, reranked_output_path + '/bert_query_by_question_answer_concat.json')
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

2021-03-11 18:43:42 - Generating BERT top-k results ...
2021-03-11 18:43:42 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_hard_faq_1.1
2021-03-11 18:43:42 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_hard_faq_1.1
2021-03-11 18:44:02 - Use pytorch device: cuda
100%|██████████| 1078/1078 [10:53<00:00,  1.65it/s]
2021-03-11 18:54:56 - Re-ranking the top-k results ...
2021-03-11 18:54:57 - Generating BERT top-k results ...
2021-03-11 18:54:57 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_hard_faq_1.1
2021-03-11 18:54:57 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_hard_faq_1.1
2021-03-11 18:54:58 - Use pytorch device: cuda
100%|██████████| 1078/1078 [19:29<00:00,  1.08s/it]
2021-03-11 19:14:29 - Re-ranking the top-k results ...
2021-03-11 19:14:30 - Generating BERT top-k results ...
2021-03-11 19:14:30 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_hard_faq_1.1
2021-03

**Re-ranking results using query_type="faq"; neg_type="simple"**

In [ ]:
# define variables
query_type="faq"; neg_type="simple"; version="1.1"; loss_type='triplet'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df
)

In [ ]:
# generate directory structure
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, reranked_output_path + '/bert_query_by_question.json')
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, reranked_output_path + '/bert_query_by_answer.json')
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, reranked_output_path + '/bert_query_by_question_answer.json')
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, reranked_output_path + '/bert_query_by_question_answer_concat.json')
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

2021-03-11 19:56:01 - Generating BERT top-k results ...
2021-03-11 19:56:01 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_simple_faq_1.1
2021-03-11 19:56:01 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_simple_faq_1.1
2021-03-11 19:56:18 - Use pytorch device: cuda
100%|██████████| 1078/1078 [10:47<00:00,  1.66it/s]
2021-03-11 20:07:06 - Re-ranking the top-k results ...
2021-03-11 20:07:07 - Generating BERT top-k results ...
2021-03-11 20:07:07 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_simple_faq_1.1
2021-03-11 20:07:07 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_simple_faq_1.1
2021-03-11 20:07:09 - Use pytorch device: cuda
100%|██████████| 1078/1078 [19:23<00:00,  1.08s/it]
2021-03-11 20:26:33 - Re-ranking the top-k results ...
2021-03-11 20:26:34 - Generating BERT top-k results ...
2021-03-11 20:26:34 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_simple_faq_1

**##################### Softmax Loss #####################**

**Re-ranking results using query_type="user_query"; neg_type="hard"**

In [ ]:
# define variables
query_type="user_query"; neg_type="hard"; version="1.1"; loss_type='softmax'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df
)

In [ ]:
# generate directory structure
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, reranked_output_path + '/bert_query_by_question.json')
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, reranked_output_path + '/bert_query_by_answer.json')
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, reranked_output_path + '/bert_query_by_question_answer.json')
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, reranked_output_path + '/bert_query_by_question_answer_concat.json')
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

2021-03-11 21:07:46 - Generating BERT top-k results ...
2021-03-11 21:07:53 - Use pytorch device: cuda
100%|██████████| 1078/1078 [08:50<00:00,  2.03it/s]
2021-03-11 21:16:44 - Re-ranking the top-k results ...
2021-03-11 21:16:45 - Generating BERT top-k results ...
2021-03-11 21:16:47 - Use pytorch device: cuda
100%|██████████| 1078/1078 [16:06<00:00,  1.12it/s]
2021-03-11 21:32:55 - Re-ranking the top-k results ...
2021-03-11 21:32:56 - Generating BERT top-k results ...
2021-03-11 21:32:58 - Use pytorch device: cuda
100%|██████████| 1078/1078 [17:02<00:00,  1.05it/s]
2021-03-11 21:50:02 - Re-ranking the top-k results ...
2021-03-11 21:50:03 - Generating BERT top-k results ...
2021-03-11 21:50:06 - Use pytorch device: cuda
100%|██████████| 1078/1078 [17:02<00:00,  1.05it/s]
2021-03-11 22:07:09 - Re-ranking the top-k results ...


**Re-ranking results using query_type="user_query"; neg_type="simple"**

In [12]:
# define variables
query_type="user_query"; neg_type="simple"; version="1.1"; loss_type='softmax'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [13]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df
)

In [ ]:
# generate directory structure
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, reranked_output_path + '/bert_query_by_question.json')
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, reranked_output_path + '/bert_query_by_answer.json')
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, reranked_output_path + '/bert_query_by_question_answer.json')
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, reranked_output_path + '/bert_query_by_question_answer_concat.json')
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

2021-03-11 22:07:10 - Generating BERT top-k results ...
2021-03-11 22:07:20 - Use pytorch device: cuda
100%|██████████| 1078/1078 [08:47<00:00,  2.04it/s]
2021-03-11 22:16:08 - Re-ranking the top-k results ...
2021-03-11 22:16:09 - Generating BERT top-k results ...
2021-03-11 22:16:11 - Use pytorch device: cuda
100%|██████████| 1078/1078 [15:59<00:00,  1.12it/s]
2021-03-11 22:32:12 - Re-ranking the top-k results ...
2021-03-11 22:32:13 - Generating BERT top-k results ...
2021-03-11 22:32:15 - Use pytorch device: cuda
 35%|███▌      | 378/1078 [06:20<12:17,  1.05s/it]

In [14]:
# generate directory structure
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, reranked_output_path + '/bert_query_by_question_answer.json')
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, reranked_output_path + '/bert_query_by_question_answer_concat.json')
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

2021-03-12 03:55:53 - Generating BERT top-k results ...
2021-03-12 03:56:00 - Use pytorch device: cuda
100%|██████████| 1078/1078 [07:31<00:00,  2.39it/s]
2021-03-12 04:03:33 - Re-ranking the top-k results ...
2021-03-12 04:03:34 - Generating BERT top-k results ...
2021-03-12 04:03:36 - Use pytorch device: cuda
100%|██████████| 1078/1078 [07:28<00:00,  2.40it/s]
2021-03-12 04:11:05 - Re-ranking the top-k results ...


**Re-ranking results using query_type="faq"; neg_type="hard"**

In [15]:
# define variables
query_type="faq"; neg_type="hard"; version="1.1"; loss_type='softmax'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [16]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df
)

In [17]:
# generate directory structure
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, reranked_output_path + '/bert_query_by_question.json')
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')
bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, reranked_output_path + '/bert_query_by_answer.json')
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, reranked_output_path + '/bert_query_by_question_answer.json')
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, reranked_output_path + '/bert_query_by_question_answer_concat.json')
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

2021-03-12 04:11:07 - Generating BERT top-k results ...
2021-03-12 04:11:14 - Use pytorch device: cuda
100%|██████████| 1078/1078 [03:53<00:00,  4.61it/s]
2021-03-12 04:15:08 - Re-ranking the top-k results ...
2021-03-12 04:15:09 - Generating BERT top-k results ...
2021-03-12 04:15:11 - Use pytorch device: cuda
100%|██████████| 1078/1078 [06:58<00:00,  2.57it/s]
2021-03-12 04:22:10 - Re-ranking the top-k results ...
2021-03-12 04:22:11 - Generating BERT top-k results ...
2021-03-12 04:22:13 - Use pytorch device: cuda
100%|██████████| 1078/1078 [07:25<00:00,  2.42it/s]
2021-03-12 04:29:40 - Re-ranking the top-k results ...
2021-03-12 04:29:41 - Generating BERT top-k results ...
2021-03-12 04:29:42 - Use pytorch device: cuda
100%|██████████| 1078/1078 [07:25<00:00,  2.42it/s]
2021-03-12 04:37:09 - Re-ranking the top-k results ...


**Re-ranking results using query_type="faq"; neg_type="simple"**

In [18]:
# define variables
query_type="faq"; neg_type="simple"; version="1.1"; loss_type='softmax'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [19]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df
)

In [20]:
# generate directory structure
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, reranked_output_path + '/bert_query_by_question.json')
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, reranked_output_path + '/bert_query_by_answer.json')
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, reranked_output_path + '/bert_query_by_question_answer.json')
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, reranked_output_path + '/bert_query_by_question_answer_concat.json')
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

2021-03-12 04:37:10 - Generating BERT top-k results ...
2021-03-12 04:37:16 - Use pytorch device: cuda
100%|██████████| 1078/1078 [03:56<00:00,  4.55it/s]
2021-03-12 04:41:14 - Re-ranking the top-k results ...
2021-03-12 04:41:14 - Generating BERT top-k results ...
2021-03-12 04:41:16 - Use pytorch device: cuda
100%|██████████| 1078/1078 [07:05<00:00,  2.53it/s]
2021-03-12 04:48:23 - Re-ranking the top-k results ...
2021-03-12 04:48:24 - Generating BERT top-k results ...
2021-03-12 04:48:26 - Use pytorch device: cuda
100%|██████████| 1078/1078 [07:35<00:00,  2.37it/s]
2021-03-12 04:56:02 - Re-ranking the top-k results ...
2021-03-12 04:56:03 - Generating BERT top-k results ...
2021-03-12 04:56:05 - Use pytorch device: cuda
100%|██████████| 1078/1078 [07:35<00:00,  2.37it/s]
2021-03-12 05:03:42 - Re-ranking the top-k results ...
